Use your chosen MLOps platform for the following -

1. Load the (same) athletes dataset.
2. With your dataset use AutoML to find the best model (if it is no-code, submit screenshots).
3. Report any data insights from AutoML run.
4. What are the reported top 5 features?
5. What are the top 3 models per validation score when using

   5.1 all features

   5.2 only the top features (if you have to choose a number put in 3)
6. What are the top 3 models per speed when using

   6.1 all features
   
   6.2 only the top features (if you have to choose a number put in 3)
7. How does the top models compare to your previously developed model (assignments 1 and 2) in terms of validation score and speed?
8. Is your platform AutoML no-code/low-code/full-code and why?
9. Repeat 2 - 5 using H2O AutoML (https://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.htmlLinks to an external site.). Submit code.

I used ChatGTP 4o to help me understand key concepts and debug codes.

### Load Data

My athletes_cleaned dataframe from assginment 2 is saved under workspace->default->athletes_cleaned in databricks. Since I will need to compare results with assginment 2, I will road the same athletes_cleaned from databricks.

In [0]:
# Force Spark session to initialize
spark

In [0]:
# Reload cleaned table from assginment 2. 
athletes_df = spark.table("workspace.default.athletes_cleaned")

---------------------------------------------------------------------------
SystemError                               Traceback (most recent call last)
File <command-5507547720733480>, line 2
      1 # Reload cleaned table from assginment 2. 
----> 2 athletes_df = spark.table("workspace.default.athletes_cleaned")

File /databricks/python_shell/lib/dbruntime/UserNamespaceInitializer.py:43, in TripWire.__getattr__(self, item)
     42 def __getattr__(self, item):
---> 43     self._raise()

File /databricks/python_shell/lib/dbruntime/UserNamespaceInitializer.py:33, in TripWire._raise(self)
     31     raise self._exc from self._exc.reason
     32 if self.msg == 'not_implemented' and self.name == 'spark':
---> 33     raise SystemError(self._UNINITIALIZED_INTERNAL_ERROR.format(self.name))
     34 if self.msg == 'not_implemented':
     35     raise NotImplementedError(self._UNSUPPORTED_SERVERLESS.format(self.name))

SystemError: Internal error: spark should be initialized with the first noteb

In [0]:
athletes_df.limit(10).display()

athlete_id,name,region,affiliate,gender,age,height,weight,eat,train,background,howlong,total_lift
21269.0,Erik Acevedo,Southern California,CrossFit Training Yard,Male,30.0,71.0,200.0,I eat whatever is convenient|,I workout mostly at a CrossFit Affiliate|I incorporate CrossFit.com workouts|,College sports,1-2 years|,1110.0
21685.0,Richard Ablett,Africa,Cape CrossFit,Male,28.0,70.0,176.0,I eat 1-3 full cheat meals per week|,I workout mostly at a CrossFit Affiliate|,No athletic background,2-4 years|,910.0
25464.0,Joe Abruzzo,North East,CrossFit Rapture,Male,35.0,68.0,225.0,I eat quality foods but don't measure the amount|,I workout mostly at a CrossFit Affiliate|I record my workouts|,Other,2-4 years|,1335.0
43767.0,Brigham Abbott,North Central,River North CrossFit,Male,36.0,71.0,199.0,I eat quality foods but don't measure the amount|,I workout mostly at a CrossFit Affiliate|I have a coach who determines my programming|I record my workouts|,College sports,1-2 years|,1354.0
55504.0,Jason Ackerman,North East,CrossFit Soulshine,Male,36.0,64.0,155.0,I eat strict Paleo|,"I workout mostly at home, work, or a traditional gym|I incorporate CrossFit.com workouts|I record my workouts|",College sports,4+ years|,1225.0
71305.0,Jerad Abbott,North Central,Koda CrossFit,Male,29.0,69.0,190.0,I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,I workout mostly at a CrossFit Affiliate|I have a coach who determines my programming|,College sports,6-12 months|,1440.0
2414.0,Bill Adams,South East,First Coast CrossFit,Male,44.0,69.0,168.0,I eat quality foods but don't measure the amount|,I workout mostly at a CrossFit Affiliate|,Other,4+ years|,1270.0
2720.0,Justin Adams,South Central,Woodward CrossFit,Male,24.0,68.0,180.0,I eat quality foods but don't measure the amount|,I workout mostly at a CrossFit Affiliate|I have a coach who determines my programming|I record my workouts|,Other,2-4 years|,1435.0
2818.0,Andy Adams,South Central,Brenham CrossFit,Male,26.0,70.0,197.0,I eat quality foods but don't measure the amount|,I workout mostly at a CrossFit Affiliate|I have a coach who determines my programming|,Other,4+ years|,1500.0
6922.0,Daniel Adamson,South West,CrossFit The Point,Male,31.0,67.0,150.0,I eat quality foods but don't measure the amount|,I workout mostly at a CrossFit Affiliate|I incorporate CrossFit.com workouts|I write my own programming|I record my workouts|,College sports,2-4 years|,1145.0


In [0]:
athletes_df.printSchema()

root
 |-- athlete_id: double (nullable = true)
 |-- name: string (nullable = true)
 |-- region: string (nullable = true)
 |-- affiliate: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- height: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- eat: string (nullable = true)
 |-- train: string (nullable = true)
 |-- background: string (nullable = true)
 |-- howlong: string (nullable = true)
 |-- total_lift: double (nullable = true)



In [0]:
print(f"Rows: {athletes_df.count()}, Columns: {len(athletes_df.columns)}")

Rows: 30015, Columns: 13


In [0]:
type(athletes_df)

pyspark.sql.connect.dataframe.DataFrame

In [0]:
athletes_df.describe().show()

+-------+------------------+---------------+-------------------+--------------------+------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+--------------------+------------------+
|summary|        athlete_id|           name|             region|           affiliate|gender|              age|           height|            weight|                 eat|               train|         background|             howlong|        total_lift|
+-------+------------------+---------------+-------------------+--------------------+------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+--------------------+------------------+
|  count|             30015|          30015|              30015|               29062| 30015|            30015|            30015|             30015|               30015|               30015|              30015|               30015|             30015|


In [0]:
# Identify categorical (string) columns
categorical_columns = [field.name for field in athletes_df.schema.fields if field.dataType.simpleString() == 'string']

# Loop through and show count of unique values and the values themselves
for col in categorical_columns:
    count = athletes_df.select(col).distinct().count()
    print(f"\nColumn: '{col}' | Distinct values: {count}")
    athletes_df.select(col).distinct().show(truncate=False)


Column: 'name' | Distinct values: 29533
+---------------------+
|name                 |
+---------------------+
|john bennett         |
|Greg Burt            |
|David Eisen          |
|erik gundersen       |
|josh anderson        |
|Rich Borucki         |
|Jared Ziesmer        |
|Mario Duarte         |
|Gregory Koehler      |
|Drew McKenna         |
|Will Richardson      |
|Terry Anderson       |
|Tom Diehl            |
|Justin Flagg         |
|Aaron Harrell        |
|James Macdonald      |
|Pierre-André Paquette|
|Dylan Frantz         |
|Drew Rykert          |
|Michael Davis        |
+---------------------+
only showing top 20 rows

Column: 'region' | Distinct values: 17
+-------------------+
|region             |
+-------------------+
|Asia               |
|Mid Atlantic       |
|South East         |
|Africa             |
|Europe             |
|South West         |
|North West         |
|Canada West        |
|North East         |
|Australia          |
|Central East       |
|Northern 

### Use Databricks AutoML to predict total_lift (a regression problem)

In [0]:
# Register athletes_df as a temporary SQL view:
athletes_df.createOrReplaceTempView("athletes_cleaned")

Since I am using databricks free edition, which does not support no-code AutoML UI, I will  run AutoML via code below. 

In [0]:
from databricks import automl

---------------------------------------------------------------------------
ImportError                               Traceback (most recent call last)
File <command-5507547720733494>, line 1
----> 1 from databricks import automl

ImportError: cannot import name 'automl' from 'databricks' (/databricks/python_shell/lib/databricks/__init__.py)

Since I do not have the paid version of databricks to use its AutoML, I will run FLAML AutoML step-by-step in Databricks Free Edition. 

In [0]:
%pip install flaml

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install --upgrade --force-reinstall flaml==2.3.5

  Using cached FLAML-2.3.5-py3-none-any.whl.metadata (16 kB)
Using cached FLAML-2.3.5-py3-none-any.whl (322 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 126.3 MB/s eta 0:00:00
  Attempting uninstall: NumPy
    Found existing installation: numpy 1.23.5
    Not uninstalling numpy at /databricks/python3/lib/python3.11/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-e1df9765-991e-484c-b8fd-999d2ee0f4c6
    Can't uninstall 'numpy'. No files were found to uninstall.
  Attempting uninstall: flaml
    Found existing installation: FLAML 2.3.5
    Uninstalling FLAML-2.3.5:
      Successfully uninstalled FLAML-2.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
databricks-connect 15.4.5 requires numpy<2,>=1.15, but you have numpy 2.3.1 which is incompatible.
scipy 1.11.1 requires numpy<1.28.0,>=1.21.6, but you have numpy 2.3

In [0]:
dbutils.library.restartPython()

In [0]:
from flaml import AutoML
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

/databricks/python/lib/python3.11/site-packages/mlflow/protos/service_pb2.py:11: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-5507547720733496>, line 1
----> 1 from flaml import AutoML
      2 from sklearn.model_selection import train_test_split
      3 from sklearn.metrics import mean_squared_error, r2_score

File /databricks/python_shell/lib/dbruntime/autoreload/discoverability/hook.py:71, in AutoreloadDiscoverabilityHook._patched_import(self, name, *args, **kwargs)
     65 if not self._should_hint and (
     66     (module := sys.modules.get(absolute_name)) is not None and
     67     (fname := get_allowed_file_name_or_none(module)) is not None and
     68     (mtime := os.stat(fname).st_mtime) > self.last_mtime_by_modname.get(
     69         absolute_name, float("inf")) and not self._should_hint):
     70     self._should_hint = True
---> 71 module = self._original_builtins_import(name, *args, **kwargs)
     72 if (fname := fname or get_allo

### Initialize H2O and Convert DataFrame

In [0]:
# Start H2O
h2o.init()

# Convert Spark DataFrame to Pandas
athletes_pd = athletes_df.toPandas()

# Convert Pandas DataFrame to H2O Frame
athletes_h2o = h2o.H2OFrame(athletes_pd)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.13" 2024-10-15 LTS; OpenJDK Runtime Environment Zulu17.54+21-CA (build 17.0.13+11-LTS); OpenJDK 64-Bit Server VM Zulu17.54+21-CA (build 17.0.13+11-LTS, mixed mode, sharing)
  Starting server from /local_disk0/.ephemeral_nfs/envs/pythonEnv-e1df9765-991e-484c-b8fd-999d2ee0f4c6/lib/python3.11/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp0do4u1vj
  JVM stdout: /tmp/tmp0do4u1vj/h2o_spark_e1df9765_991e_484c_b8fd_99_started_from_python.out
  JVM stderr: /tmp/tmp0do4u1vj/h2o_spark_e1df9765_991e_484c_b8fd_99_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ...................... failed.


---------------------------------------------------------------------------
H2OConnectionError                        Traceback (most recent call last)
File /local_disk0/.ephemeral_nfs/envs/pythonEnv-e1df9765-991e-484c-b8fd-999d2ee0f4c6/lib/python3.11/site-packages/h2o/h2o.py:270, in init(url, ip, port, name, https, cacert, insecure, username, password, cookies, proxy, start_h2o, nthreads, ice_root, log_dir, log_level, max_log_file_size, enable_assertions, max_mem_size, min_mem_size, strict_version_check, ignore_config, extra_classpath, jvm_custom_args, bind_to_localhost, verbose, **kwargs)
    269 try:
--> 270     h2oconn = H2OConnection.open(url=url, ip=ip, port=port, name=name, https=https,
    271                                  verify_ssl_certificates=verify_ssl_certificates, cacert=cacert,
    272                                  auth=auth, proxy=proxy, cookies=cookies, verbose=verbose,
    273                                  msgs=("Checking whether there is an H2O instance run

Can not use databricks free edition to finish the tasks at all. I will switch to Colab. 